In [1]:
import re
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

# Uploading Data

In [2]:
# File Paths
accepted_filepath = "Data/accepted_2007_to_2018Q4.csv"
rejected_filepath = "Data/rejected_2007_to_2018Q4.csv"

In [3]:
# reading in csv and converting to dataframes
accepted_df = pd.read_csv(accepted_filepath, error_bad_lines=False,low_memory=False)
rejected_df = pd.read_csv(rejected_filepath, error_bad_lines=False,low_memory=False)

In [4]:
list(accepted_df)

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'last_fico_range_high',
 'last_fico_range_low',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 '

In [5]:
list(rejected_df)

['Amount Requested',
 'Application Date',
 'Loan Title',
 'Risk_Score',
 'Debt-To-Income Ratio',
 'Zip Code',
 'State',
 'Employment Length',
 'Policy Code']

In [6]:
print(rejected_df['Loan Title'].unique())
print('-------------------')
print(accepted_df['purpose'].unique())
print('-------------------')
print(accepted_df['title'].unique())

['Wedding Covered but No Honeymoon' 'Consolidating Debt'
 'Want to consolidate my debt' ... 'dougie03' 'freeup'
 'Business Advertising Loan']
-------------------
['debt_consolidation' 'small_business' 'home_improvement' 'major_purchase'
 'credit_card' 'other' 'house' 'vacation' 'car' 'medical' 'moving'
 'renewable_energy' 'wedding' 'educational' nan]
-------------------
['Debt consolidation' 'Business' nan ... 'takeitaway' 'Creditt Card Loan'
 'debt reduction/hone updates']


## Trimming data for Model 1

Model 1 - Classification Model - Combining matching columns of Accepted and Rejected Data - Use that to train a model that can predict Acceptance/Rejection. 

In [7]:
# select Needed columns
trim_accept_df = accepted_df[[
    'loan_amnt',
    'issue_d',
    'fico_range_high',
    'dti',
    'zip_code',
    'addr_state',
    'emp_length',
    'purpose'
]]

In [8]:
# select Needed columns
trim_reject_df = rejected_df[['Amount Requested',
 'Application Date',
 'Risk_Score',
 'Debt-To-Income Ratio',
 'Zip Code',
 'State',
 'Employment Length',
 'Loan Title']]

In [9]:
trim_accept_df.shape

(2260701, 8)

In [10]:
trim_reject_df.shape

(27648741, 8)

In [12]:
# adding a target value to the data
trim_accept_df['Loan Status'] = "Accepted"
trim_accept_df.head()

C:\Users\pamponsah\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,loan_amnt,issue_d,fico_range_high,dti,zip_code,addr_state,emp_length,purpose,Loan Status
0,3600.0,Dec-2015,679.0,5.91,190xx,PA,10+ years,debt_consolidation,Accepted
1,24700.0,Dec-2015,719.0,16.06,577xx,SD,10+ years,small_business,Accepted
2,20000.0,Dec-2015,699.0,10.78,605xx,IL,10+ years,home_improvement,Accepted
3,35000.0,Dec-2015,789.0,17.06,076xx,NJ,10+ years,debt_consolidation,Accepted
4,10400.0,Dec-2015,699.0,25.37,174xx,PA,3 years,major_purchase,Accepted


In [13]:
# adding a target value to the data
trim_reject_df['Loan Status'] = "Rejected"
trim_reject_df.head()

,Amount Requested,Application Date,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Loan Title,Loan Status
0,1000.0,2007-05-26,693.0,10%,481xx,NM,4 years,Wedding Covered but No Honeymoon,Rejected
1,1000.0,2007-05-26,703.0,10%,010xx,MA,< 1 year,Consolidating Debt,Rejected
2,11000.0,2007-05-27,715.0,10%,212xx,MD,1 year,Want to consolidate my debt,Rejected
3,6000.0,2007-05-27,698.0,38.64%,017xx,MA,< 1 year,waksman,Rejected
4,1500.0,2007-05-27,509.0,9.43%,209xx,MD,< 1 year,mdrigo,Rejected


### More clean up and writing to CSV - to be used for Model 1 (get team's opinion on whether we'll need more variables)

In [14]:
print(trim_reject_df.info())
print(trim_accept_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27648741 entries, 0 to 27648740
Data columns (total 9 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount Requested      float64
 1   Application Date      object 
 2   Risk_Score            float64
 3   Debt-To-Income Ratio  object 
 4   Zip Code              object 
 5   State                 object 
 6   Employment Length     object 
 7   Loan Title            object 
 8   Loan Status           object 
dtypes: float64(2), object(7)
memory usage: 1.9+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Data columns (total 9 columns):
 #   Column           Dtype  
---  ------           -----  
 0   loan_amnt        float64
 1   issue_d          object 
 2   fico_range_high  float64
 3   dti              float64
 4   zip_code         object 
 5   addr_state       object 
 6   emp_length       object 
 7   purpose          object 
 8   Loan Status      object 
dt

In [15]:
# recast accepted issue date as date
trim_accept_df['issue_d'] = pd.to_datetime(trim_accept_df['issue_d'])
trim_accept_df.head()

C:\Users\pamponsah\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,loan_amnt,issue_d,fico_range_high,dti,zip_code,addr_state,emp_length,purpose,Loan Status
0,3600.0,2015-12-01,679.0,5.91,190xx,PA,10+ years,debt_consolidation,Accepted
1,24700.0,2015-12-01,719.0,16.06,577xx,SD,10+ years,small_business,Accepted
2,20000.0,2015-12-01,699.0,10.78,605xx,IL,10+ years,home_improvement,Accepted
3,35000.0,2015-12-01,789.0,17.06,076xx,NJ,10+ years,debt_consolidation,Accepted
4,10400.0,2015-12-01,699.0,25.37,174xx,PA,3 years,major_purchase,Accepted


#### this is where we can decide what year(s) to focus on

In [16]:
# filter for 2 years? Can do more if needed
accept_filtered_df = trim_accept_df[trim_accept_df['issue_d'].dt.year.between(2017,2018)]

In [17]:
# verify
accept_filtered_df['issue_d'].dt.year.unique()

array([2018, 2017], dtype=int64)

In [18]:
# recast rejected application date as date
trim_reject_df['Application Date'] = pd.to_datetime(trim_reject_df['Application Date'])
trim_reject_df.head()

,Amount Requested,Application Date,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Loan Title,Loan Status
0,1000.0,2007-05-26,693.0,10%,481xx,NM,4 years,Wedding Covered but No Honeymoon,Rejected
1,1000.0,2007-05-26,703.0,10%,010xx,MA,< 1 year,Consolidating Debt,Rejected
2,11000.0,2007-05-27,715.0,10%,212xx,MD,1 year,Want to consolidate my debt,Rejected
3,6000.0,2007-05-27,698.0,38.64%,017xx,MA,< 1 year,waksman,Rejected
4,1500.0,2007-05-27,509.0,9.43%,209xx,MD,< 1 year,mdrigo,Rejected


In [19]:
# filter for 2 years? Can do more if needed
reject_filtered_df = trim_reject_df[trim_reject_df['Application Date'].dt.year.between(2017,2018)]
reject_filtered_df['Application Date'].dt.year.unique() # verify

array([2017, 2018], dtype=int64)

## Write Model 1 data to CSV

In [21]:
# write to CSV to save space for working model
accept_filtered_df.to_csv('Model_1 - ACC_REJ/acc_df_filtered.csv',index=False)

In [22]:
# write to CSV to save space for working model
reject_filtered_df.to_csv('Model_1 - ACC_REJ/rej_df_filtered.csv',index=False)

# Trimming data for Model 2

In [23]:
accepted_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# recast accepted issue date as date
accepted_df['issue_d'] = pd.to_datetime(accepted_df['issue_d'])

In [25]:
# filter for 2 years? Can do more if needed
acc_filtered_all_df = accepted_df[accepted_df['issue_d'].dt.year.between(2017,2018)]

In [26]:
acc_filtered_all_df.shape

(938821, 151)

# Write Model 2 data to CSV

In [14]:
# write to CSV to save space for working model 
acc_df_filtered.to_csv('Model_2 - ACC_Predicts/acc_df_filtered_all.csv',index=False)